In [2]:
import pandas as pd
from scipy.stats.mstats import winsorize
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from scipy import stats
from scipy.stats import shapiro
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

# csv 넣는 폴더 고정해주세요

In [3]:
df = pd.read_csv('./dataset/label지정.csv', encoding='CP949')                        ## 성휘 파일 경로
# df = pd.read_excel('../eeez/전처리파일/label지정.xlsx', index_col=0)                ## 재인 파일 경로
# df = pd.read_csv('../데이터자료/label지정.csv', encoding='CP949')                   ## 의영 파일 경로
# df = pd.read_excel('./csv/dateset2_label최종.xlsx', index_col=0)                   ## 경록 파일 경로

### 윈저라이징

In [4]:
df.drop(['회사명', '거래소코드', '회계년도_x', '상장일', '상장폐지일', '산업명', '소속코드','year', 'month','label'],axis=1,inplace=True)
for col in df.columns:
    df[col] = winsorize(df[col], limits= 0.01)

# MDA

## Bartlett 검정

### F-test 검정

In [5]:
close = df[df['label']== 1] #Existing Customer
normal = df[df['label']== 0] #Attrited Customer

In [6]:
a = []
for i in df:
    # print(i)
    # lresult = stats.levene(close[i], normal[i])
    lresult = stats.bartlett(close[i], normal[i])
    
    # print('LeveneResult(F) : %.3f \np-value : %.3f' % (lresult))

    # result = stats.ttest_ind(close[i], normal[i], equal_var=False) #등분산
    # print('t statistic : %.3f \np-value : %.3f' % (result))
    # a.append([i,lresult[-1],result[-1]])
    a.append([i,lresult[-1]])

c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:2303: RuntimeWarning: divide by zero encountered in log
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)
c:\ProgramData\Anaconda3\lib\site-packages\scipy\stats\morestats.py:2303: RuntimeWarning: invalid value encountered in double_scalars
  numer = (Ntot*1.0 - k) * log(spsq) - np.sum((Ni - 1.0)*log(ssq), axis=0)


In [7]:
a= pd.DataFrame(a)
a.columns=["피처값",'F-test']
a


,피처값,F-test
0,[제조]매출액증가율,8.096743e-08
1,[제조]유형자산증가율,8.912869e-05
2,[제조]비유동자산증가율,3.487216e-01
3,[제조]총자본증가율,1.864021e-01
4,[제조]자기자본증가율,1.071900e-14
5,[제조]유동자산증가율,1.701051e-04
6,[제조]순이익증가율,6.966236e-03
7,[제조]재고자산증가율,1.335022e-05
8,[제조]매출액총이익률,1.261631e-04
9,[제조]매출액순이익률,1.070479e-80


### 등분산과 이분산 구분

In [8]:
a["분산"] =''
a["T-test"] =""
for i in a.index:
    if a.loc[i,"F-test"]>=0.05:
        a.loc[i,"분산"] = "homo"
    else:
        a.loc[i,"분산"] = "hetero"
a

,피처값,F-test,분산,T-test
0,[제조]매출액증가율,8.096743e-08,hetero,
1,[제조]유형자산증가율,8.912869e-05,hetero,
2,[제조]비유동자산증가율,3.487216e-01,homo,
3,[제조]총자본증가율,1.864021e-01,homo,
4,[제조]자기자본증가율,1.071900e-14,hetero,
5,[제조]유동자산증가율,1.701051e-04,hetero,
6,[제조]순이익증가율,6.966236e-03,hetero,
7,[제조]재고자산증가율,1.335022e-05,hetero,
8,[제조]매출액총이익률,1.261631e-04,hetero,
9,[제조]매출액순이익률,1.070479e-80,hetero,


### 등분산 이분산 T-test
- 등분산일시 T-test
- 이분산일시 Welch T-test

In [9]:
a["분산"] =''
a["T-test"] =""
for i in a.index:
    if a.loc[i,"F-test"]>=0.05:
        a.loc[i,"분산"] = "homo"
        result = stats.ttest_ind(close[a.loc[i,"피처값"]], normal[a.loc[i,"피처값"]], equal_var=True)       ## equal_var = True Student T-test
        a.loc[i,"T-test"] = result[-1]
    else:
        a.loc[i,"분산"] = "hetero"
        result = stats.ttest_ind(close[a.loc[i,"피처값"]], normal[a.loc[i,"피처값"]], equal_var=False)      ## equal_var = False Welchs T-Test
        a.loc[i,"T-test"] = result[-1]
a


,피처값,F-test,분산,T-test
0,[제조]매출액증가율,8.096743e-08,hetero,0.000452
1,[제조]유형자산증가율,8.912869e-05,hetero,0.146287
2,[제조]비유동자산증가율,3.487216e-01,homo,0.0
3,[제조]총자본증가율,1.864021e-01,homo,0.0
4,[제조]자기자본증가율,1.071900e-14,hetero,0.000008
5,[제조]유동자산증가율,1.701051e-04,hetero,0.000015
6,[제조]순이익증가율,6.966236e-03,hetero,0.221912
7,[제조]재고자산증가율,1.335022e-05,hetero,0.0
8,[제조]매출액총이익률,1.261631e-04,hetero,0.0
9,[제조]매출액순이익률,1.070479e-80,hetero,0.0


### T-test 결과값이 0.05이하인 유의미한 피처 선정

In [10]:
bartlett =a[a["T-test"]<=0.05]
# bartlett=bartlett.sort_values(['T-test'],ascending=True)
# bartlett = bartlett.loc[[1,2,3,5,7,10,11,14,15,21,26,30,33,35,37]]
bartlett


,피처값,F-test,분산,T-test
0,[제조]매출액증가율,8.096743e-08,hetero,0.000452
2,[제조]비유동자산증가율,3.487216e-01,homo,0.0
3,[제조]총자본증가율,1.864021e-01,homo,0.0
4,[제조]자기자본증가율,1.071900e-14,hetero,0.000008
5,[제조]유동자산증가율,1.701051e-04,hetero,0.000015
7,[제조]재고자산증가율,1.335022e-05,hetero,0.0
8,[제조]매출액총이익률,1.261631e-04,hetero,0.0
9,[제조]매출액순이익률,1.070479e-80,hetero,0.0
10,[제조]총자본사업이익률,5.545513e-18,hetero,0.0
11,[제조]총자본순이익률,4.036610e-45,hetero,0.0


### 피처별 범주 분류

In [11]:
bartlett['범주']=''

<ipython-input-11-e55ebdfce24a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bartlett['범주']=''


In [22]:
bartlett['범주'][[0,2,3,4,5,7]] = '성장성'
bartlett['범주'][[8,9,10,11,12,13,14]] = '수익성'
bartlett['범주'][[15,17,20,21,23,26,27,28]] = '활동성'
bartlett['범주'][[29,30,31,32,33,34,35,36,37,47]] = '안정성'
bartlett['범주'][[38,39,40,41,42,44,45]] = '생산성'

<ipython-input-22-a2721579f8e0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bartlett['범주'][[0,2,3,4,5,7]] = '성장성'
<ipython-input-22-a2721579f8e0>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bartlett['범주'][[8,9,10,11,12,13,14]] = '수익성'
<ipython-input-22-a2721579f8e0>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bartlett['범주'][[15,17,20,21,23,26,27,28]] = '활동성'
<ipython-input-22-a2721579f8e0>:4: SettingWithCopyWarning: 
A v

In [24]:
Grow_bartlett = bartlett[bartlett['범주']=='성장성']
Profit_bartlett = bartlett[bartlett['범주']=='수익성']
Activity_bartlett = bartlett[bartlett['범주']=='활동성']
Stability_bartlett = bartlett[bartlett['범주']=='안정성']
Productivity_bartlett = bartlett[bartlett['범주']=='생산성']

### 범주별 피처

In [25]:
Grow_bartlett       ## 성장성 피처

,피처값,F-test,분산,T-test,범주
0,[제조]매출액증가율,8.096743e-08,hetero,0.000452,성장성
2,[제조]비유동자산증가율,3.487216e-01,homo,0.0,성장성
3,[제조]총자본증가율,1.864021e-01,homo,0.0,성장성
4,[제조]자기자본증가율,1.071900e-14,hetero,0.000008,성장성
5,[제조]유동자산증가율,1.701051e-04,hetero,0.000015,성장성
7,[제조]재고자산증가율,1.335022e-05,hetero,0.0,성장성


In [26]:
Profit_bartlett     ## 수익성 피처

,피처값,F-test,분산,T-test,범주
8,[제조]매출액총이익률,1.261631e-04,hetero,0.0,수익성
9,[제조]매출액순이익률,1.070479e-80,hetero,0.0,수익성
10,[제조]총자본사업이익률,5.545513e-18,hetero,0.0,수익성
11,[제조]총자본순이익률,4.036610e-45,hetero,0.0,수익성
12,[제조]자기자본순이익률,1.878436e-87,hetero,0.0,수익성
13,[제조]1주당순이익,6.309810e-02,homo,0.0,수익성
14,[제조]1주당매출액,5.874191e-12,hetero,0.0,수익성


In [27]:
Activity_bartlett   ## 활동성 피처

,피처값,F-test,분산,T-test,범주
15,[제조]총자본회전률,1.382090e-01,homo,0.0,활동성
17,[제조]비유동자산회전률,2.322028e-07,hetero,0.0,활동성
20,[제조]매입채무회전률,6.612722e-94,hetero,0.0,활동성
21,[제조]매입채무회전기간,6.893020e-27,hetero,0.0,활동성
23,[제조]매출채권회전기간,1.320025e-21,hetero,0.000327,활동성
26,[제조]순운전자본회전률,6.821750e-02,homo,0.0,활동성
27,[제조]유동자산회전률,9.444850e-01,homo,0.00154,활동성
28,[제조]당좌자산회전률,2.660212e-01,homo,0.001091,활동성


In [28]:
Stability_bartlett  ## 안정성 피처

,피처값,F-test,분산,T-test,범주
29,[제조]유동비율,4.549235e-09,hetero,0.0,안정성
30,[제조]당좌비율,1.398865e-08,hetero,0.0,안정성
31,[제조]현금비율,7.499112e-08,hetero,0.0,안정성
32,[제조]부채비율,1.765822e-97,hetero,0.0,안정성
33,[제조]순운전자본비율,1.733802e-03,hetero,0.0,안정성
34,[제조]자기자본배율,6.538218e-33,hetero,0.0,안정성
35,[제조]비유동비율,1.125987e-72,hetero,0.0,안정성
36,[제조]유동자산구성비율,1.227852e-01,homo,0.000042,안정성
37,[제조]자기자본구성비율,1.399577e-05,hetero,0.0,안정성
47,[제조]이자보상배율(이자비용),8.892149e-22,hetero,0.000614,안정성


In [29]:
Productivity_bartlett   ## 생산성 피처

,피처값,F-test,분산,T-test,범주
38,[제조]부가가치,7.452751e-04,hetero,0.0,생산성
39,[제조]총자본투자효율,6.512239e-40,hetero,0.0,생산성
40,[제조]설비투자효율,1.340890e-01,homo,0.0,생산성
41,[제조]기계투자효율,2.207324e-156,hetero,0.0,생산성
42,[제조]부가가치율,1.246357e-87,hetero,0.0,생산성
44,[제조]자본분배율,9.919202e-16,hetero,0.00034,생산성
45,[제조]이윤분배율,1.149550e-18,hetero,0.00307,생산성


# Bartlett VIF 확인

In [30]:
bartlett['피처값'].tolist()

['[제조]매출액증가율',
 '[제조]비유동자산증가율',
 '[제조]총자본증가율',
 '[제조]자기자본증가율',
 '[제조]유동자산증가율',
 '[제조]재고자산증가율',
 '[제조]매출액총이익률',
 '[제조]매출액순이익률',
 '[제조]총자본사업이익률',
 '[제조]총자본순이익률',
 '[제조]자기자본순이익률',
 '[제조]1주당순이익',
 '[제조]1주당매출액',
 '[제조]총자본회전률',
 '[제조]비유동자산회전률',
 '[제조]매입채무회전률',
 '[제조]매입채무회전기간',
 '[제조]매출채권회전기간',
 '[제조]순운전자본회전률',
 '[제조]유동자산회전률',
 '[제조]당좌자산회전률',
 '[제조]유동비율',
 '[제조]당좌비율',
 '[제조]현금비율',
 '[제조]부채비율',
 '[제조]순운전자본비율',
 '[제조]자기자본배율',
 '[제조]비유동비율',
 '[제조]유동자산구성비율',
 '[제조]자기자본구성비율',
 '[제조]부가가치',
 '[제조]총자본투자효율',
 '[제조]설비투자효율',
 '[제조]기계투자효율',
 '[제조]부가가치율',
 '[제조]자본분배율',
 '[제조]이윤분배율',
 '[제조]이자보상배율(이자비용)']

In [31]:
def vif(data):
    import pandas as pd
    from statsmodels.stats.outliers_influence import variance_inflation_factor

    # VIF 출력을 위한 데이터 프레임 형성
    vif = pd.DataFrame()

    # VIF 값과 각 Feature 이름에 대해 설정
    vif["VIF Factor"] = [variance_inflation_factor(data.values, i) for i in range(len(data.columns))]
    vif["features"] = data.columns 

    # VIF 값이 높은 순으로 정렬
    vif = vif.sort_values(by="VIF Factor", ascending=False)
    vif = vif.reset_index().drop(columns='index')
    
    return vif

vif(df[['[제조]매출액증가율',
 '[제조]비유동자산증가율',
 '[제조]총자본증가율',
 '[제조]자기자본증가율',
 '[제조]유동자산증가율',
 '[제조]재고자산증가율',
 '[제조]매출액총이익률',
 '[제조]매출액순이익률',
 '[제조]총자본사업이익률',
 '[제조]총자본순이익률',
 '[제조]자기자본순이익률',
 '[제조]1주당순이익',
 '[제조]1주당매출액',
 '[제조]총자본회전률',
 '[제조]비유동자산회전률',
 '[제조]매입채무회전률',
 '[제조]매입채무회전기간',
 '[제조]매출채권회전기간',
 '[제조]순운전자본회전률',
 '[제조]유동자산회전률',
 '[제조]당좌자산회전률',
 '[제조]유동비율',
 '[제조]당좌비율',
 '[제조]현금비율',
 '[제조]부채비율',
 '[제조]순운전자본비율',
 '[제조]자기자본배율',
 '[제조]비유동비율',
 '[제조]유동자산구성비율',
 '[제조]자기자본구성비율',
 '[제조]부가가치',
 '[제조]총자본투자효율',
 '[제조]설비투자효율',
 '[제조]기계투자효율',
 '[제조]부가가치율',
 '[제조]자본분배율',
 '[제조]이윤분배율',
 '[제조]이자보상배율(이자비용)']])

,VIF Factor,features
0,86.302563,[제조]순운전자본회전률
1,84.937490,[제조]유동비율
2,76.263409,[제조]당좌비율
3,70.495862,[제조]유동자산회전률
4,62.036960,[제조]유동자산구성비율
5,58.813339,[제조]총자본회전률
6,51.266599,[제조]자기자본구성비율
7,29.866513,[제조]비유동비율
8,24.237789,[제조]당좌자산회전률
9,21.044035,[제조]부채비율
